In [131]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os


In [132]:
load_dotenv()
OPENSEA_API = os.getenv('OPENSEA_API_KEY')

In [133]:
# Function to fetch data from an API
def fetch_api_data(url, headers=None):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data from {url}")
        return {}


In [134]:
# USER CHOOSES COLLECTION
collection_slug = input("Enter the collection: ")
url = f"https://api.opensea.io/api/v2/events/collection/{collection_slug}"

# API's
url1 = f"https://api.opensea.io/api/v2/collections/{collection_slug}/stats"
url2 = f"https://api.opensea.io/api/v2/collections/{collection_slug}"
url3 = f"https://api.opensea.io/api/v2/events/collection/{collection_slug}"

headers1 = {"accept": "application/json", "x-api-key": OPENSEA_API}
headers2 = {"accept": "application/json", "x-api-key": OPENSEA_API}
headers3 = {"accept": "application/json", "x-api-key": OPENSEA_API}

data1 = fetch_api_data(url1, headers1)
data2 = fetch_api_data(url2, headers2)
data3 = fetch_api_data(url3, headers3)

In [135]:
# Process and convert data from the first API
df1_total = pd.DataFrame([data1['total']])
df1_intervals = pd.DataFrame(data1['intervals'])

In [136]:
# Process and convert data from the second API
collection_fields = ['collection', 'name', 'description', 'image_url', 'banner_image_url', 'owner', 'category', 'is_disabled', 'is_nsfw', 'trait_offers_enabled', 'collection_offers_enabled', 'opensea_url', 'project_url', 'wiki_url', 'discord_url', 'telegram_url', 'twitter_username', 'instagram_username', 'total_supply', 'created_date']
df2_collection = pd.DataFrame([{field: data2.get(field, '') for field in collection_fields}])
df2_contracts = pd.DataFrame(data2.get('contracts', []))
df2_fees = pd.DataFrame(data2.get('fees', []))
df2_payment_tokens = pd.DataFrame(data2.get('payment_tokens', []))
df2_rarity = pd.DataFrame([data2.get('rarity', {})])

In [137]:
events_normalized = []
for event in data3['asset_events']:
    # Initialize event_data with event fields, excluding nft, asset, payment, criteria
    event_data = {k: v for k, v in event.items() if k not in ['nft', 'asset', 'payment', 'criteria']}
    
    # Check and add nft data if present
    if 'nft' in event and event['nft'] is not None:
        nft_data = {'nft_' + k: v for k, v in event['nft'].items()}
        event_data.update(nft_data)
    
    # Check and add asset data if present
    elif 'asset' in event and event['asset'] is not None:
        asset_data = {'asset_' + k: v for k, v in event['asset'].items()}
        event_data.update(asset_data)
    
    # Check and include payment data if necessary and present
    if 'payment' in event and event['payment'] is not None:
        payment_data = {'payment_' + k: v for k, v in event['payment'].items()}
        event_data.update(payment_data)
    
    events_normalized.append(event_data)

df_events = pd.DataFrame(events_normalized)

In [138]:
# Export to CSV
df1_total.to_csv('api1_total_stats.csv', index=False)
df1_intervals.to_csv('api1_intervals_stats.csv', index=False)
df2_collection.to_csv('api2_collection_info.csv', index=False)
df2_contracts.to_csv('api2_collection_contracts.csv', index=False)
df2_fees.to_csv('api2_collection_fees.csv', index=False)
df2_payment_tokens.to_csv('api2_collection_payment_tokens.csv', index=False)
df2_rarity.to_csv('api2_collection_rarity.csv', index=False)
df_events.to_csv('api3_events.csv', index=False)


In [139]:
print(df_events.head())

  event_type                                         order_hash   order_type  \
0      order  0xee450499348f8ce7cb4b1830ebc7c89d6f246192c5b7...  trait_offer   
1      order  0x5398b7f831b4e7ad13d2f1257d62f8e4ba89e34d43bd...   item_offer   
2      order  0x348e61a5b003692e32329b448127025120609a215012...   item_offer   
3      order  0xb8aa80258051148fdf13d608f50c2c570a06569f8fc2...   item_offer   
4      order  0x878728288697297f99eed1d3ec65aa5d8527e0cc3385...   item_offer   

      chain                            protocol_address  start_date  \
0  ethereum  0x00000000000000adc04c56bf30ac9d3c0aaf14dc  1709171238   
1  ethereum  0x00000000000000adc04c56bf30ac9d3c0aaf14dc  1709171237   
2  ethereum  0x00000000000000adc04c56bf30ac9d3c0aaf14dc  1709171237   
3  ethereum  0x00000000000000adc04c56bf30ac9d3c0aaf14dc  1709171230   
4  ethereum  0x00000000000000adc04c56bf30ac9d3c0aaf14dc  1709171230   

   expiration_date  quantity                                       maker  \
0       17091718

In [140]:
print(df1_total.head())

          volume  sales  average_price  num_owners    market_cap  floor_price  \
0  418454.421694  68881       6.075034        9558  23795.219937        1.195   

  floor_price_symbol  
0                ETH  


In [141]:
print(df1_intervals.head())

     interval       volume  volume_diff  volume_change  sales  sales_diff  \
0     one_day    50.300299        -24.0      -0.311876     39   -0.380952   
1   seven_day   635.643253        -67.0      -0.138518    522    0.000000   
2  thirty_day  3173.367667       -520.0      -0.150553   2529    0.000000   

   average_price  
0       1.289751  
1       1.217707  
2       1.254791  


In [142]:
print(df2_fees)

   fee                                   recipient  required
0  2.5  0x0000a26b00c1f0df003000390027140000faa719      True
1  5.0  0xe65b6865dbce299ae6a20efcc7543362540741d8     False


In [143]:
print(df2_rarity)


  strategy_id strategy_version               calculated_at  max_rank  \
0  openrarity          0.7.1b0  2022-11-15T03:32:42.716033     19422   

   tokens_scored  
0          19422  


In [144]:
print(df2_payment_tokens)


  symbol                                     address     chain  \
0    ETH  0x0000000000000000000000000000000000000000  ethereum   
1   WETH  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  ethereum   

                                 image           name  decimals  \
0   https://raw.seadn.io/files/ETH.svg          Ether        18   
1  https://raw.seadn.io/files/WETH.svg  Wrapped Ether        18   

           eth_price                usd_price  
0  1.000000000000000  3420.650000000000091000  
1  1.000000000000000  3422.150000000000091000  


In [145]:
print(df2_collection)


  collection                          name  \
0     clonex  CLONE X - X TAKASHI MURAKAMI   

                                         description  \
0  🧬 CLONE X 🧬\r\n\r\n20,000 next-gen Avatars, by...   

                                           image_url  \
0  https://i.seadn.io/gae/XN0XuD8Uh3jyRWNtPTFeXJg...   

                                    banner_image_url  \
0  https://i.seadn.io/s/raw/files/21d546f6ace1227...   

                                        owner category  is_disabled  is_nsfw  \
0  0x12ea19217c65f36385bb030d00525c1034e2f0af     pfps        False    False   

   trait_offers_enabled  collection_offers_enabled  \
0                  True                       True   

                            opensea_url           project_url wiki_url  \
0  https://opensea.io/collection/clonex  http://www.rtfkt.com            

                discord_url telegram_url twitter_username instagram_username  \
0  https://discord.gg/rtfkt                         RTFKT            

In [146]:
print(df2_contracts)


                                      address     chain
0  0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b  ethereum
